<a href="https://colab.research.google.com/github/kerryqliu/stats170/blob/irene/tf_idf_with_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import nltk
import re
from sqlalchemy import create_engine

In [73]:
# ENGINE

In [4]:
query = "SELECT * from fullDocuments f join documentLabels d on f.docid = d.docid;"
fulldocs = pd.read_sql_query(query, con=engine)
fulldocs

,docid,fulltext,docid,caseid,label
0,1.0,Document Text: Example 1\nReferring Doctor: Un...,1,-1,1.0
1,2.0,Document Text: Example 2\nProgress Notes\nDate...,2,-1,1.0
2,3.0,Document Text: Example 3\nChief Complaint:\n1....,3,-1,4.0
3,4.0,Document Text: 14\nse\nLAIDA\n*\nPatient Infor...,4,11594,1.0
4,5.0,Document Text: The first set of documents behi...,5,11594,1.0
...,...,...,...,...,...
110,111.0,Document Text: 9:10\nFAX\n2002/041\n'\nPatient...,111,87832,1.0
111,112.0,Document Text: # 23/24\nfined VI\nvyy\nScan on...,112,88178,1.0
112,113.0,Document Text: 15:03 From\nPage: 1/24\nNUFACTO...,113,95139,1.0
113,114.0,Document Text: 03:58:42 p.m.\n1/2\nNU FACTOR\n...,114,95739,2.0


In [34]:
corpus_dict = {}
corpus = []
tags = []
for i, row in fulldocs.iterrows():
  corpus.append(row['fulltext'])
  tags.append(row['label'])
  corpus_dict[i] = (row['fulltext'], row['label'])

In [65]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(corpus, tags, test_size=0.2, random_state=10)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [60]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train, labels_train)
features_train = selector.transform(features_train).toarray()
features_test = selector.transform(features_test).toarray()

In [61]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, StratifiedKFold

In [63]:
model = LogisticRegression(C=100).fit(features_train, labels_train)
score = model.score(features_test, labels_test)
print(score)

0.4782608695652174


Adjusting C, which determines the amount of regularization.
Lower values increase regularization

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}
tfidf_search = GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)

In [55]:
Cs = [1e-7, 1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]

for c in Cs:
  curr_model = LogisticRegression(C=c).fit(features_train, labels_train)
  curr_score = curr_model.score(features_test, labels_test)
  print(f'{c}: {curr_score}')

1e-07: 0.43478260869565216
1e-05: 0.43478260869565216
0.001: 0.43478260869565216
0.1: 0.43478260869565216
1.0: 0.43478260869565216
10.0: 0.4782608695652174
100.0: 0.4782608695652174


highest accuracy:
max_df = 0.5 or without this, C = 100, 0.4782608695652174

considering different parameters:
- penalty
- C
- class_weight
- solver: 'newton-cg', 'sag', 'saga', 'lbfgs'
- max_iter

with bigrams

In [70]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(corpus, tags, test_size=0.2, random_state=10)

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (1,2))
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [72]:
for c in Cs:
  curr_model = LogisticRegression(C=c).fit(features_train, labels_train)
  curr_score = curr_model.score(features_test, labels_test)
  print(f'{c}: {curr_score}')

1e-07: 0.43478260869565216
1e-05: 0.43478260869565216
0.001: 0.43478260869565216
0.1: 0.43478260869565216
1.0: 0.43478260869565216
10.0: 0.43478260869565216
100.0: 0.4782608695652174
